## Project (E) 448
## Linear Regression - Evidence Approximation
### Daniël S. van der Westhuizen
### 17158680

In [1]:
import numpy as np
import pylab as pl
from numpy.linalg import inv
from functions import *

I will attempt to use the evidence approximation method to estimate the hyperparameters using the same data supplied for the one-dimensional predictive distribution linear regression demonstration.

In [2]:
N = 25
x = np.linspace(0,1,N)
#x = [ 0,0.45,0.55,1]
t = np.zeros(N)
for a in range(0,N):
    t[a] = np.sin(x[a]*2*np.pi) + np.random.normal(0,0.2) + x[a]
    

In [3]:
s=0.2
mu = np.linspace(0,2,9)
phi_vector = ['0','0','0','0','0','0','0','0','0']
def phi(x,mu):
    phi = np.exp(-((x-mu)**2)/(2*(s**2)))
    return phi

In [4]:
alpha = 1
beta = 5

In [5]:
design_matrix = np.zeros((N,9))
for a in range(0,9):
    for b in range(0,N):
        design_matrix[b,a] = phi(x,mu[a])[b]
S_N = inv( alpha*(np.identity(9)) + beta*np.dot(design_matrix.transpose(),design_matrix) )
m_N = beta * (   np.dot( np.dot(S_N,design_matrix.transpose()), np.atleast_2d(t).transpose() ) )

$ \lambda_{\textit{i}} $ is the eigenvalues of $ \Phi^{T} \Phi $ from equation (3.81)

$$ \gamma = \sum_{i}^{} \frac{\lambda_{\textit{i}}}{\alpha + \lambda_\textit{i}}   \qquad  (3.91) $$

$$ \textbf{m}_N = \beta \textbf{S}_N \Phi^{\textsf{T}} \textbf{t} \qquad (3.53) $$

$$ \alpha = \frac{\gamma}{\textbf{m}_N^T \textbf{m}_N}   \qquad (3.92)  $$

$$ \frac{1}{\beta} = \frac{1}{N-\gamma} \sum_{n=1}^{N}\{ t_n - \textbf{m}_N^T \phi(\textsf{x}_n) \}^2  \qquad (3.95)  $$

I will start with $\alpha = 1$ and $\beta = 1$ and iteratively evaluate these two parameters with (3.92) and (3.95)  until they converge.

In [6]:
alpha = 1
beta = 1

eigenvalues = np.linalg.eig( np.dot( design_matrix.transpose(), design_matrix) )[0]
print("eigenvalues: ", eigenvalues)

N_iter = 50 #Number of iterations

for a in range(0,N_iter):
    
    lambda_var = 0
    for b in range(0,9):
        lambda_var += eigenvalues[b] / (alpha + eigenvalues[b])
        
    m_N = beta * ( np.dot( np.dot(S_N,design_matrix.transpose()),
                          np.atleast_2d(t).transpose() ) )
        
    alpha = lambda_var / np.dot( m_N.transpose(), m_N)[0,0]
    
    sum_term = 0    #first we'll evaluate the sigma factor in equation (3.95)
    
    for b in range(0,N):
        phi_vector_pred = np.zeros(9)
        for a in range(0,9):
            phi_vector_pred[a] = phi(x[b],mu[a])
        phi_vector_pred = np.atleast_2d(phi_vector_pred).transpose()
        sum_term += ( t[a] - np.dot( m_N.transpose() , phi_vector_pred ) )**2
        
    beta = 1 / (  ( 1 / ( N - lambda_var ) ) * sum_term[0,0]  )
    
print("lambda = ", lambda_var)
print("alpha = ", alpha)
print("beta = ",beta)

eigenvalues:  [  1.95336891e+01   1.04639874e+01   3.80718140e+00   9.80485731e-01
   2.12282749e-01   1.18246368e-02   9.64839647e-06   2.20486236e-10
   1.64895065e-16]
lambda =  1.37740316414
alpha =  12.7002358904
beta =  0.980361918871


There must be something wrong with either the code or my understanding of the mathematics because these hyperparameter values does not work as well as the ones I have selected.